# LAGO Results

# Initialization

## Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import seaborn as sns
import os
from IPython.display import display, Math, Latex
%matplotlib inline

In [2]:
import glob
import json
import sys
import re
sys.path.append('../')
import utils_src.ml_utils as ml_utils
from multiprocessing import Pool
import contextlib

## Variables initialization

In [3]:
# path = os.path.expanduser('~/www/UAI2018')

In [4]:
base_results_dir_pattern = os.path.expanduser(r'output/test/%s')
base_results_dir = dict(CUB=base_results_dir_pattern%'CUB*',
                        AWA2=base_results_dir_pattern%'AWA2*',
                        SUN=base_results_dir_pattern%'SUN*')

In [5]:
params_to_display = ['use_trainval_set', 'model_name', 'model_variant', 'initial_learning_rate', 'max_epochs', 'LG_beta', 'LG_lambda',
                'SG_psi', 'LG_norm_groups_to_1', 'SG_num_K', 'SG_trainable', 'SG_gain', 'SG_gamma_lr', 'attributes_weight', 'repeat']
params_rename = {
    'use_trainval_set':'isTest', 
    'model_name':'Model',
    'model_variant':'ModelType',
    'initial_learning_rate':'lr', 
    'max_epochs':'mxEp', 
    'LG_beta':'β',     
    'LG_lambda':'λ', 
    'LG_norm_groups_to_1':'muExNorm', 
    'SG_psi':'ψ',
    'SG_num_K':'K', 
    'SG_trainable':'is_SG', 
    'SG_gain':'SGgain', 
    'SG_gamma_lr':'SGlr',
    'attributes_weight':'aW',
    'repeat':'rpt'}


## Function initialization

In [6]:
def to_dec_point_str(x):
    if isinstance(x, str):
        x = re.sub('(\d+)(_)(\d+)', '\g<1>.\g<3>', x )
    return x

def result_to_df(rf):
    """
    Converts a single (json) result file to a dataframe
    """
    # Load result file
#     print('fname = ', rf)
    res = pd.read_json(rf,  typ='series')
#     print('res dict = ', res)

    # Extract only metric keys
    metric_keys = [k for k in res.keys() if 'metric_' in k]

    # Take only the metric results
    df_out = 100 * res[metric_keys]
        
    hp_k, hp_v = zip(*[(params_rename[param], res[param]) for param in params_to_display])
    df_out = df_out.append(pd.Series(hp_v, index=hp_k))
    
    # Convert string numeric values to numbers
    df_out = df_out.apply(to_dec_point_str)
    df_out = pd.to_numeric(df_out, errors='ignore')
    df_out['fname'] = rf
    return df_out

def load_results_to_df(base_results_dir, verbose=True):
    if verbose:
        ml_utils.tic()

    # Find result files
    rfiles = ml_utils.run_bash(f'find {base_results_dir} -name "metric_results.json"').split('\n')

    # Load results to a list
    num_pool_workers=1 # can be bigger than 1, to enable parallel execution
    with contextlib.closing(Pool(num_pool_workers)) as po: # This ensures that the processes get closed once they are done
         pool_results = po.map_async(result_to_df, (rf for rf in rfiles))
         results_list = pool_results.get()

    # Aggregate results to a dataframe

    # First find ordered column names over a small subset of results
    # This stage is performed because adding rows one-by-one does not ruin the order in case a new column is added
    df = pd.DataFrame(columns=results_list[0].keys())
    results_list_subset = [results_list[k] for k in range(0, len(results_list), max(int(len(results_list)/200.), 1))]
    for res in results_list_subset:
        # Append current results record to the DataFrame
        df = df.append(res, ignore_index=True)
    cols = df.columns

    # Then aggregate all results and reorder columns
    df = pd.DataFrame(columns=cols)

    df = df.append(results_list, ignore_index=True)
    metric_keys = [k for k in df.columns if 'metric_' in k]

    # Final conversion of string numeric values to numbers
    df = df.apply(pd.to_numeric, errors="ignore")
    # Drop irrelevant metric
    if 'metric_val_accuracy' in df.columns:
        df = df.drop(['metric_val_accuracy'], axis=1)

    # Fill NA
    df = df.fillna('NA')

    if verbose:
        ml_utils.toc()

    return df, metric_keys

In [7]:
def allbut(names, columns):
     names = set(names)
     return [item for item in columns if item not in names ]

def df_query_stats(qry, df, metric_keys, query2=None, min_counts=None, best_only=False, verbose=False):
    qry = qry.replace(',', ' and ')
    main_metric = 'metric_val_zs_accuracy'
    df0 = df.copy().drop(['fname'], axis=1)
    repeatitions_keys = ['rpt', 'tcSD', 'imSD']
    repeatitions_keys = list(filter(lambda k: k in df.columns, repeatitions_keys))
    df1 = df0.query(qry)
    if query2 is not None:
        df1 = df1.query(query2)

    dfgroup = df1.groupby(allbut(metric_keys + repeatitions_keys, df1.columns)) #.query('nD>=63 and lr>0.1e-5')
    # Evaluate mean over repeatitions per experiment
    dfmean = dfgroup.mean().round(2).drop(repeatitions_keys, axis=1)
    dfstd = dfgroup.std().round(4).drop(repeatitions_keys, axis=1)
    N = len(dfmean)
    sorted_ix = np.argsort(dfmean.loc[:, main_metric])[::-1]

    dfsem = dfstd/(dfgroup.count().drop(repeatitions_keys, axis=1)**0.5)
    counts_df = dfgroup.count().drop(repeatitions_keys, axis=1)

    dfmean = dfmean.iloc[sorted_ix]
    dfsem = dfsem.iloc[sorted_ix]
    dfstd = dfstd.iloc[sorted_ix]
    counts_df = counts_df.iloc[sorted_ix]

    if min_counts is not None:
        min_counts_bool = (counts_df[main_metric] >= min_counts).values

        dfmean = dfmean.iloc[min_counts_bool]
        dfsem = dfsem.iloc[min_counts_bool]
        dfstd = dfstd.iloc[min_counts_bool]
        counts_df = counts_df.iloc[min_counts_bool]

    if best_only:
        dfmean = dfmean.iloc[0:1]
        dfsem = dfsem.iloc[0:1]
        dfstd = dfstd.iloc[0:1]
        counts_df = counts_df.iloc[0:1]
    df_counts = counts_df

    if verbose:
        display(dfmean.iloc[0:min(N,10)])
        print('\n\nCounts:')
        display(counts_df.iloc[0:min(N,10)])
        print('\n\nStd Error Of Mean:')
        display(dfsem.iloc[0:min(N,10)])
        print('STD:')
        display(dfstd.iloc[0:min(N,10)])
        print('path:\n', df.query(qry)['fname'].iloc[0])
    return dfmean, dfstd, dfsem, df_counts

In [8]:
def pprint_experiments(df_experiments, ignore_NA=True):
    for _, e in df_experiments.reset_index().T.to_dict().items():
        if ignore_NA:
            print(dict((k,v) for k,v in e.items() if v!='NA'))
        else:
            print(e)

# Load results

In [9]:
df_cub, metric_keys = load_results_to_df(base_results_dir['CUB'])

Elapsed time is 0.39540958404541016 seconds.


In [10]:
df_awa2, _ = load_results_to_df(base_results_dir['AWA2'])

Elapsed time is 0.3545973300933838 seconds.


In [11]:
df_sun, _ = load_results_to_df(base_results_dir['SUN'])

Elapsed time is 0.3908424377441406 seconds.


In [12]:
# Define params for collecting results
name = r'LAGO'


Singletons = 'LAGO-Singletons'
KSoft = 'LAGO-KSoft'
SemanticHard = 'LAGO-SemanticHard'
SemanticSoft = 'LAGO-SemanticSoft'

err_df = 'dfsem'

# Aggregate results from soft groups experiments

In [13]:
# @ test set
df_table12 = pd.DataFrame(columns=['CUB', 'AWA2', 'SUN'])
df_table12.loc[f'{KSoft}', :]    = [None, None, None]
df_table12.loc[f'{SemanticSoft}', :]  = [None, None, None]
df_table12.loc[f'{SemanticHard}', :]  = [None, None, None]
df_table12.loc[f'{Singletons}', :]  = [None, None, None]
df_table12.columns.name = 'accuracy (%) @test'
df_table12_err = df_table12.copy()*np.nan
df_table12_values = df_table12.copy()*np.nan

## SUN

### Semantic Soft

In [14]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", lr==1e-4, β==1e-4, λ==1e-4, muExNorm==1, is_SG==1, ψ==0, SGgain==3, SGlr==1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_sun, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);


print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{SemanticSoft}', 'SUN'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticSoft}', 'SUN'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticSoft}', 'SUN'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,130,0.0001,0.0001,0.0,1,-1.0,1,3.0,1.0,0.0,36.23,48.76




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,130,0.0001,0.0001,0.0,1,-1.0,1,3.0,1.0,0.0,0.414254,0.531334




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,130,0.0001,0.0001,0.0,1,-1.0,1,3.0,1.0,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_SemanticSoft', 'lr': 0.0001, 'mxEp': 130, 'β': 0.0001, 'λ': 0.0001, 'ψ': 0.0, 'muExNorm': 1, 'K': -1.0, 'is_SG': 1, 'SGgain': 3.0, 'SGlr': 1.0, 'aW': 0.0, 'metric_train_zs_accuracy': 36.23, 'metric_val_zs_accuracy': 48.76}


### K-Soft groups

In [15]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_KSoft", lr==1e-4, β==1e-4, λ==0, muExNorm==0, K==40, is_SG==1, SGgain==10, SGlr==0.1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_sun, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{KSoft}', 'SUN'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{KSoft}', 'SUN'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{KSoft}', 'SUN'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.0001,39,0.0001,0.0,0.0,0,40.0,1,10.0,0.1,1.0,53.87,57.46




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.0001,39,0.0001,0.0,0.0,0,40.0,1,10.0,0.1,1.0,0.080901,0.176515




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.0001,39,0.0001,0.0,0.0,0,40.0,1,10.0,0.1,1.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_KSoft', 'lr': 0.0001, 'mxEp': 39, 'β': 0.0001, 'λ': 0.0, 'ψ': 0.0, 'muExNorm': 0, 'K': 40.0, 'is_SG': 1, 'SGgain': 10.0, 'SGlr': 0.1, 'aW': 1.0, 'metric_train_zs_accuracy': 53.87, 'metric_val_zs_accuracy': 57.46}


### Semantic Hard

In [16]:
# Test set
# Note is_SG==0 indicates that this is a SemanticHard variant
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", is_SG==0, 2e-5<lr<4e-5, β==1e-3, λ==1e-4, muExNorm==1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_sun, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{SemanticHard}', 'SUN'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticHard}', 'SUN'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticHard}', 'SUN'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,159,0.001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,25.5,47.21




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,159,0.001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,0.025894,0.055544




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,159,0.001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_SemanticSoft', 'lr': 3.0000000000000004e-05, 'mxEp': 159, 'β': 0.001, 'λ': 0.0001, 'ψ': 0.0, 'muExNorm': 1, 'K': -1.0, 'is_SG': 0, 'SGgain': 10.0, 'SGlr': -1.0, 'aW': 0.0, 'metric_train_zs_accuracy': 25.5, 'metric_val_zs_accuracy': 47.21}


### Singletons

In [17]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="Singletons", 2e-5<lr<4e-5, β==1e-8, λ==1e-8, muExNorm==0, is_SG==0, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_sun, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{Singletons}', 'SUN'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{Singletons}', 'SUN'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{Singletons}', 'SUN'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.00003,124,1.000000e-08,1.000000e-08,0.0,0,-1.0,0,10.0,-1.0,0.0,58.42,57.14




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.00003,124,1.000000e-08,1.000000e-08,0.0,0,-1.0,0,10.0,-1.0,0.0,0.042441,0.088951




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.00003,124,1.000000e-08,1.000000e-08,0.0,0,-1.0,0,10.0,-1.0,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'Singletons', 'lr': 3.0000000000000004e-05, 'mxEp': 124, 'β': 1e-08, 'λ': 1e-08, 'ψ': 0.0, 'muExNorm': 0, 'K': -1.0, 'is_SG': 0, 'SGgain': 10.0, 'SGlr': -1.0, 'aW': 0.0, 'metric_train_zs_accuracy': 58.42, 'metric_val_zs_accuracy': 57.14}


## AWA2

### Semantic Soft

In [18]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", 2e-6<lr<4e-6, β==1e-3, λ==1e-3, muExNorm==1, is_SG==1, ψ==1e-4, SGgain==1, SGlr==0.1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_awa2, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);


print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{SemanticSoft}', 'AWA2'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticSoft}', 'AWA2'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticSoft}', 'AWA2'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,63,0.001,0.001,0.0001,1,-1.0,1,1.0,0.1,0.0,84.37,64.7




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,63,0.001,0.001,0.0001,1,-1.0,1,1.0,0.1,0.0,0.166542,0.365329




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,63,0.001,0.001,0.0001,1,-1.0,1,1.0,0.1,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_SemanticSoft', 'lr': 3e-06, 'mxEp': 63, 'β': 0.001, 'λ': 0.001, 'ψ': 0.0001, 'muExNorm': 1, 'K': -1.0, 'is_SG': 1, 'SGgain': 1.0, 'SGlr': 0.1, 'aW': 0.0, 'metric_train_zs_accuracy': 84.37, 'metric_val_zs_accuracy': 64.7}


### K-Soft groups

In [19]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_KSoft", 2e-6<lr<4e-6, β==1e-4, λ==1e-6, muExNorm==0, K==30, is_SG==1, SGgain==3, SGlr==0.01, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_awa2, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{KSoft}', 'AWA2'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{KSoft}', 'AWA2'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{KSoft}', 'AWA2'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.000003,90,0.0001,0.000001,0.0,0,30.0,1,3.0,0.01,0.0,91.89,58.84




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.000003,90,0.0001,0.000001,0.0,0,30.0,1,3.0,0.01,0.0,0.029919,0.428162




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.000003,90,0.0001,0.000001,0.0,0,30.0,1,3.0,0.01,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_KSoft', 'lr': 3e-06, 'mxEp': 90, 'β': 0.0001, 'λ': 1e-06, 'ψ': 0.0, 'muExNorm': 0, 'K': 30.0, 'is_SG': 1, 'SGgain': 3.0, 'SGlr': 0.01, 'aW': 0.0, 'metric_train_zs_accuracy': 91.89, 'metric_val_zs_accuracy': 58.84}


### Semantic-Hard

In [20]:
# Test set
# Note is_SG==0 indicates that this is a semanticHard variant
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", is_SG==0, lr==3e-6, β==0, λ==1e-5, muExNorm==1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_awa2, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{SemanticHard}', 'AWA2'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticHard}', 'AWA2'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticHard}', 'AWA2'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,55,0.0,0.00001,0.0,1,-1.0,0,10.0,-1.0,0.0,81.85,61.49




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,55,0.0,0.00001,0.0,1,-1.0,0,10.0,-1.0,0.0,0.014624,0.073969




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.000003,55,0.0,0.00001,0.0,1,-1.0,0,10.0,-1.0,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_SemanticSoft', 'lr': 3e-06, 'mxEp': 55, 'β': 0.0, 'λ': 1e-05, 'ψ': 0.0, 'muExNorm': 1, 'K': -1.0, 'is_SG': 0, 'SGgain': 10.0, 'SGlr': -1.0, 'aW': 0.0, 'metric_train_zs_accuracy': 81.85, 'metric_val_zs_accuracy': 61.49}


### Singletons

In [21]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="Singletons", 2e-6<lr<4e-6, β==1e-5, λ==1e-5, muExNorm==0, is_SG==0, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_awa2, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

df_table12_values.loc[f'{Singletons}', 'AWA2'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{Singletons}', 'AWA2'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{Singletons}', 'AWA2'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)


Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.000003,38,0.00001,0.00001,0.0,0,-1.0,0,10.0,-1.0,0.0,89.14,63.46




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.000003,38,0.00001,0.00001,0.0,0,-1.0,0,10.0,-1.0,0.0,0.023345,0.154781




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.000003,38,0.00001,0.00001,0.0,0,-1.0,0,10.0,-1.0,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'Singletons', 'lr': 3e-06, 'mxEp': 38, 'β': 1e-05, 'λ': 1e-05, 'ψ': 0.0, 'muExNorm': 0, 'K': -1.0, 'is_SG': 0, 'SGgain': 10.0, 'SGlr': -1.0, 'aW': 0.0, 'metric_train_zs_accuracy': 89.14, 'metric_val_zs_accuracy': 63.46}


## CUB

### Semantic Soft

In [22]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", 2e-5<lr<4e-5, β==1e-3, λ==1e-4, muExNorm==1, is_SG==1, ψ==1e-4, SGgain==3, SGlr==0.1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_cub, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

K = int(dfmean.reset_index().loc[0, 'K'])
df_table12_values.loc[f'{SemanticSoft}', 'CUB'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticSoft}', 'CUB'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticSoft}', 'CUB'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,100,0.001,0.0001,0.0001,1,-1.0,1,3.0,0.1,0.0,88.21,58.02




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,100,0.001,0.0001,0.0001,1,-1.0,1,3.0,0.1,0.0,0.104648,0.39158




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.00003,100,0.001,0.0001,0.0001,1,-1.0,1,3.0,0.1,0.0,5,5


{'isTest': 1, 'Model': 'LAGO', 'ModelType': 'LAGO_SemanticSoft', 'lr': 3.0000000000000004e-05, 'mxEp': 100, 'β': 0.001, 'λ': 0.0001, 'ψ': 0.0001, 'muExNorm': 1, 'K': -1.0, 'is_SG': 1, 'SGgain': 3.0, 'SGlr': 0.1, 'aW': 0.0, 'metric_train_zs_accuracy': 88.21, 'metric_val_zs_accuracy': 58.02}


### K-Soft groups

In [23]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_KSoft", 2e-5<lr<4e-5, β==1e-5, λ==1e-5, muExNorm==0, K==30, is_SG==1, SGgain==1, SGlr==1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_cub, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

K = int(dfmean.reset_index().loc[0, 'K'])
df_table12_values.loc[f'{KSoft}', 'CUB'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{KSoft}', 'CUB'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{KSoft}', 'CUB'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
# pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.00003,160,0.00001,0.00001,0.0,0,30.0,1,1.0,1.0,0.0,95.67,56.09




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.00003,160,0.00001,0.00001,0.0,0,30.0,1,1.0,1.0,0.0,0.043201,0.392519




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_KSoft,0.00003,160,0.00001,0.00001,0.0,0,30.0,1,1.0,1.0,0.0,5,5


### Semantic-Hard

In [24]:
# Test set
# Note is_SG==0 indicates that this is a semanticHard variant
qry = 'isTest==1, Model=="LAGO", ModelType=="LAGO_SemanticSoft", is_SG==0, lr==1e-4, β==1e-6, λ==1e-4, muExNorm==1, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_cub, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

K = int(dfmean.reset_index().loc[0, 'K'])
df_table12_values.loc[f'{SemanticHard}', 'CUB'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{SemanticHard}', 'CUB'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{SemanticHard}', 'CUB'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
# pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,55,0.000001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,94.8,58.36




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,55,0.000001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,0.045929,0.100399




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,LAGO_SemanticSoft,0.0001,55,0.000001,0.0001,0.0,1,-1.0,0,10.0,-1.0,0.0,5,5


### Singletons

In [25]:
# Test set
qry = 'isTest==1, Model=="LAGO", ModelType=="Singletons", lr==1e-4, β==0, λ==1e-7, muExNorm==0, is_SG==0, rpt!="NA"'
dfmean, dfstd, dfsem, df_counts = df_query_stats(qry, df_cub, metric_keys, verbose=False, query2='rpt<5',  min_counts=0, best_only=False);

print('Average')
display(dfmean)
print('\n\nS.E.M')
display(dfsem)
print('\n\nRepeatition Count')
display(df_counts)

K = int(dfmean.reset_index().loc[0, 'K'])
df_table12_values.loc[f'{Singletons}', 'CUB'] = dfmean.iloc[0]['metric_val_zs_accuracy']
df_table12.loc[f'{Singletons}', 'CUB'] ='%2.1f'%(dfmean.iloc[0]['metric_val_zs_accuracy'])
df_table12_err.loc[f'{Singletons}', 'CUB'] = locals()[err_df].iloc[0]['metric_val_zs_accuracy']
# pprint_experiments(dfmean)

Average


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.0001,58,0.0,1.000000e-07,0.0,0,-1.0,0,10.0,-1.0,0.0,99.65,54.61




S.E.M


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.0001,58,0.0,1.000000e-07,0.0,0,-1.0,0,10.0,-1.0,0.0,0.010286,0.161042




Repeatition Count


,,,,,,,,,,,,,,metric_train_zs_accuracy,metric_val_zs_accuracy
isTest,Model,ModelType,lr,mxEp,β,λ,ψ,muExNorm,K,is_SG,SGgain,SGlr,aW,,
1,LAGO,Singletons,0.0001,58,0.0,1.000000e-07,0.0,0,-1.0,0,10.0,-1.0,0.0,5,5


## Aggregated result

The following table reproduces the results reported in Table A.1 in the Supplamentary (up to standard error of mean)

In [26]:
df_table12.loc['-------------------------------', :] = '', '', ''
df_table12.loc['LAGO (CROSS-VAL)', :] = \
  df_table12.loc[SemanticSoft, 'CUB'], \
  df_table12.loc[SemanticSoft, 'AWA2'], \
  df_table12.loc[KSoft,        'SUN']
display(df_table12)

accuracy (%) @test,CUB,AWA2,SUN
LAGO-KSoft,56.1,58.8,57.5
LAGO-SemanticSoft,58.0,64.7,48.8
LAGO-SemanticHard,58.4,61.5,47.2
LAGO-Singletons,54.6,63.5,57.1
-------------------------------,,,
LAGO (CROSS-VAL),58.0,64.7,57.5


The next table shows the standard error of the mean (S.E.M)

In [29]:
display(df_table12_err)

accuracy (%) @test,CUB,AWA2,SUN
LAGO-KSoft,0.392519,0.428162,0.176515
LAGO-SemanticSoft,0.39158,0.365329,0.531334
LAGO-SemanticHard,0.100399,0.0739691,0.0555439
LAGO-Singletons,0.161042,0.154781,0.0889508
